In [ ]:
import pandas as pd
import numpy as np
from bff_processor.data_tools import regex_select, get_files, make_df
from bff_processor.utils import time_func, hist2unc, linear, parabola, heaviside, significance, power_func, apply_multiple_filters
from bff_processor.bff_meta import preselection, band_cut, isin, identity
from bff_processor.plotting_utils import produce_bff_hists, boost_plot, boost_plot2d, unc_plot
from bff_processor.bff_cuts import *
from bff_processor.sig_op_func import *
import boost_histogram as bh
import matplotlib.pyplot as plt
import mplhep as hep
hep.set_style(hep.style.CMS)
import dill as pickle

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    })

In [ ]:
# get <era> files
era = 2016
file_re = regex_select(era)
file_dict = get_files(file_re)

In [ ]:
#make df of background
backgrounds = file_dict['DY']+file_dict['TT']+file_dict['ST']+file_dict['VB']
background_df = preselection()(make_df(backgrounds))

In [ ]:
#make df of signals
import re
bff_dict = {}
for fname in file_dict['BFF']:
    #print(fname)
    name = re.findall('_M_([0-9]+)_dbs(\d)p(\d+)', fname)[0]
    name = "{} {}.{}".format(*name)
    #print(name)
    if name not in bff_dict:
        bff_dict[name] = []
    bff_dict[name].append(fname)
for name, item in bff_dict.items():
    if len(item) != 1: continue
    print(item)
    bff_dict[name] = preselection()(make_df(item))
bff_dict.keys()

In [ ]:
bff_samples = ['125 0.04', '150 0.04', '200 0.04', '250 0.04', '300 0.04', '350 0.04','400 0.04','450 0.04','500 0.04']

In [ ]:
SR1_nom = lambda df: df[df.SR1_nom==1]
SR2_nom = lambda df: df[df.SR2_nom==1]

In [ ]:
popt, fig  = plot_opt_sig('RelMET_nom',bff_dict, background_df, bff_samples, power_func, filter_func=SR1_nom, p0=param_dict['popt_relmet_1'], minimize_func=False)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'RelMET_nom', "_one_jet"), transparent=False)
popt, fig  = plot_opt_sig('RelMET_nom',bff_dict, background_df, bff_samples, power_func, filter_func=SR2_nom, p0=param_dict['popt_relmet_2'], minimize_func=False)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'RelMET_nom', "_two_jet"), transparent=False)

popt, fig  = plot_opt_sig('HTLT_nom',bff_dict, background_df, bff_samples, linear, filter_func=RelMET_SR1_nom, p0=param_dict['popt_htlt_1'], minimize_func=False)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'HTLT_nom', "_one_jet"), transparent=False)
popt, fig  = plot_opt_sig('HTLT_nom',bff_dict, background_df, bff_samples, linear, filter_func=RelMET_SR2_nom, p0=param_dict['popt_htlt_2'], minimize_func=False)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'HTLT_nom', "_two_jet"), transparent=False)

popt, fig  = plot_opt_sig('TMB_nom',bff_dict, background_df, bff_samples, heaviside, filter_func=HTLT_SR1_nom, p0=param_dict['popt_TMB_1'], minimize_func=False, direction=-1)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'TMB_nom', "_one_jet"), transparent=False)
popt, fig  = plot_opt_sig('TMB_nom',bff_dict, background_df, bff_samples, heaviside, filter_func=HTLT_SR2_nom, p0=param_dict['popt_TMB_2'], minimize_func=False, direction=-1)
fig.savefig("output/sig_tuning/{}_{}_{}.pdf".format(era, 'TMB_nom', "_two_jet"), transparent=False)